In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow==2.10.0


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow import keras

# Vérification de la version de TensorFlow
print(f"TensorFlow version: {tf.__version__}")

# Charger le modèle préentraîné
try:
    custom_objects = {'Conv2DTranspose': keras.layers.Conv2DTranspose}
    model = load_model('/content/roads_extraction.h5', compile=False, custom_objects=custom_objects)
    print("Modèle chargé avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {e}")

# Fonction de prétraitement des images
def preprocess_image(image_path, target_size=(256, 256)):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"L'image {image_path} est introuvable ou corrompue.")
    img = cv2.resize(img, target_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Fonction pour superposer le masque sur l'image originale
def overlay_mask_on_image(image, mask, alpha=0.5):
    mask_colored = np.zeros_like(image)
    mask_colored[:, :, 2] = mask  # Appliquer la couleur rouge
    overlaid_image = cv2.addWeighted(image, 1 - alpha, mask_colored, alpha, 0)
    return overlaid_image

# Inférence sur un répertoire de données
def infer_directory(input_dir, output_dir, model, target_size=(256, 256), threshold=0.5):
    mask_dir = os.path.join(output_dir, "masques")
    overlay_dir = os.path.join(output_dir, "images_segmentées")

    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(overlay_dir, exist_ok=True)

    for file_name in os.listdir(input_dir):
        image_path = os.path.join(input_dir, file_name)
        try:
            # Prétraitement et prédiction
            input_image = preprocess_image(image_path, target_size)
            pred = model.predict(input_image, verbose=0)[0, :, :, 0]
            binary_mask = (pred > threshold).astype(np.uint8) * 255

            # Charger l'image originale
            original_image = cv2.imread(image_path)
            original_image = cv2.resize(original_image, target_size)

            # Générer l'image segmentée
            segmented_image = overlay_mask_on_image(original_image, binary_mask)

            # Sauvegarder le masque et l'image segmentée
            mask_output_path = os.path.join(mask_dir, file_name)
            overlay_output_path = os.path.join(overlay_dir, file_name)

            cv2.imwrite(mask_output_path, binary_mask)
            cv2.imwrite(overlay_output_path, segmented_image)
            print(f"Résultats sauvegardés pour {file_name}")
        except Exception as e:
            print(f"Erreur sur {file_name}: {e}")

# Exécution de l'inférence
input_directory = '/content/drive/MyDrive/BDD-7227'
output_directory = '/content/drive/MyDrive/Resultat_segmentation_Unet'

infer_directory(input_directory, output_directory, model,threshold=0.3)


TensorFlow version: 2.10.0
Modèle chargé avec succès!
Résultats sauvegardés pour route4.png
Résultats sauvegardés pour route3.png
Résultats sauvegardés pour route2.png
Résultats sauvegardés pour route1.png
Résultats sauvegardés pour route5.png
Résultats sauvegardés pour route6.png
